# Notebook to test quotes and trades in CTrader

In [ ]:
# python
from datetime import datetime
import json
import numpy as np
from pprint import pformat
import time

# our modules
from ctrader.ctrader import CTrader
# be careful to include from demo
from configs.ctrader_demo import (HOST, SENDER_COMP_ID, PASSWORD, CURRENCY, CLIENT_ID, DEBUG)

# allow to use an updated module and use the change directly by refreshing the cell
# without having to restart the entire notebook
%load_ext autoreload
%autoreload 2

In [ ]:
# check
HOST, SENDER_COMP_ID, PASSWORD, CURRENCY, CLIENT_ID, DEBUG

In [ ]:
if "live" in SENDER_COMP_ID:
    print("You are using a LIVE account! Use a demo account for this test!")
    assert False

In [ ]:
# build trader object
api = CTrader(
    server=HOST,
    account=SENDER_COMP_ID,
    password=PASSWORD,
    currency=CURRENCY,
    client_id=CLIENT_ID,
    debug=DEBUG,
)
api

In [ ]:
# Check the connection status
api.isconnected()

In [ ]:
api.client

In [ ]:
api.fix

In [ ]:
api.market_data_list

In [ ]:
# logout
# api.logout()

# Only market orders to buy and sell without TP nor SL, then close

Also no subscription to prices for this.

In [ ]:
api.buy("XAUUSD", 0.02)

In [ ]:
api.buy("EURUSD", 0.01)

In [ ]:
api.buy("CADCHF", 0.01)

In [ ]:
api.sell("USDCHF", 0.30)

In [ ]:
api.sell("XTIUSD", 0.30)

In [ ]:
api.sell("US30", 1)

In [ ]:
# looking directly at positions does not see the changes
api.positions()

In [ ]:
# to see the changes we need to log out and log back in
# logout
api.logout()
# time.sleep(1)
# Check the connection status
print(api.isconnected())
# build trader object
api = CTrader(
    server=HOST,
    account=SENDER_COMP_ID,
    password=PASSWORD,
    currency=CURRENCY,
    client_id=CLIENT_ID,
    debug=DEBUG,
)
time.sleep(1)
#
positions = api.positions()
print(pformat(positions))

In [ ]:
# close all positions of a given symbol
api.close("USDCHF")

In [ ]:
api.close("CADCHF")

In [ ]:
api.close("EURUSD")

In [ ]:
api.close_all()

# Real-time quote

In [ ]:
symbol = "EURUSD"
symbol

In [ ]:
# subscribe to symbols
# api.subscribe(["BTCUSD", "ETHUSD", "EURUSD", "GBPUSD", "EURUSD", "GBPUSD", "AUDUSD", "CADCHF", "XAUUSD"])
# api.subscribe(["EURUSD", "GBPUSD"])
# api.subscribe(["XAUUSD", "AUDUSD"])
api.subscribe([symbol])

In [ ]:
# wait one second for the quote to arrive
time.sleep(1)
# list of quotes for all symbols
quote = api.quote(symbol)
print(quote)

In [ ]:
type(quote[symbol])

In [ ]:
api.fix.spot_market_request("XAUUSD")

In [ ]:
api.fix.spot_price_list

In [ ]:
# Convert Unix timestamp to datetime object
timestamp_ms = quote[symbol]["time"] # in ms, needs to be converted in seconds
timestamp_s = timestamp_ms/1000.0
dt = datetime.fromtimestamp(timestamp_s)
dt

In [ ]:
# quote for a single symbol
# this does not work
# quote = api.quote(symbol)
# print(quote)

In [ ]:
api.client

# List positions

In [ ]:
api.positions()

In [ ]:
# logout
api.logout()
# build trader object
api = CTrader(
    server=HOST,
    account=SENDER_COMP_ID,
    password=PASSWORD,
    currency=CURRENCY,
    client_id=CLIENT_ID,
    debug=DEBUG,
)
#
positions = api.positions()
print(pformat(positions))

In [ ]:
for position in positions:
    print()
    print("New position:")
    print(pformat(position))

# List orders 

It is about limit and stop orders, ss market orders get executed right away anyway.

In [ ]:
api.orders()

In [ ]:
orders = api.orders()
print(pformat(orders))

In [ ]:
for order in orders:
    print()
    print("New order:")
    print(pformat(order))

# Close positions

In [ ]:
position

In [ ]:
# Close position by id of all amount by giving it the entire amount the position
api.positionCloseById(position['pos_id'], position['amount'])

In [ ]:
# Close position by id and and give the partial position to close in lots
api.positionClosePartial(position['pos_id'], volume=0.01)

In [ ]:
# close all positions
api.close_all()

In [ ]:
api.close_all_positions_of_symbol("AUDUSD")

In [ ]:
api.positions()

In [ ]:
for position in api.positions():
    if position[name] != symbol:
        continue
    # if here close the entire position
    # Close position by id of all amount by giving it the entire amount the position
    api.positionCloseById(position['pos_id'], position['amount'])

# Close Orders

In [ ]:
order

In [ ]:
api.orderCancelById(order["ord_id"])

In [ ]:
# close all orders
api.cancel_all()

# Place buy and sell orders: Market, Limit and Stop

In [ ]:
if False:
    # logout
    api.logout()
    # build trader object
    api = CTrader(
        server=HOST,
        account=SENDER_COMP_ID,
        password=PASSWORD,
        currency=CURRENCY,
        client_id=CLIENT_ID,
        debug=DEBUG,
    )
#
symbol = "EURUSD"
volume = 0.15
pips = 20
# subscribe to get the current price (but you get only the latest)
api.subscribe([symbol])
# wait one second for the price to arrive
time.sleep(1)
# Current price
quote = api.quote(symbol)
print(quote)

price_bid = quote['bid'] 
price_ask = quote['ask']
spread = price_ask - price_bid
# 
below = round(price_bid - pips/10_000, 6)
above = round(price_ask + pips/10_000, 6)
# limit buy: price must be below current price
limit_price_buy = price_bid - pips/10_000
# limit sell: price must be above current price
limit_price_sell = price_ask + pips/10_000
# stop buy: price must be above current price
stop_price_buy = price_ask + pips/10_000
# stop sell: price must be below current price
stop_price_sell = price_bid - pips/10_000
print(f"price bid={price_bid}, ask={price_ask}, spread={spread}, limit_price buy={limit_price_buy}, sell={limit_price_sell}, stop_price buy={stop_price_buy}, sell={stop_price_sell}")


In [ ]:
quote = api.quote()
quote

In [ ]:
volume=0.06
id = api.buyMarket(symbol, volume, stoploss=None, takeprofit=above, price=price_ask)
print(f"Position: {id}")

In [ ]:
id = api.sellMarket(symbol, volume, stoploss=None, takeprofit=None, price=price_bid)
print(f"Position: {id}")

In [ ]:
# market orders without SL, TP or entry price
volume=0.01
id = api.buyMarket(symbol, volume, stoploss=None, takeprofit=None, price=None)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=None, takeprofit=None, price=None)
print(f"Position: {id}")

# market orders with only entry price
volume=0.02
id = api.buyMarket(symbol, volume, stoploss=None, takeprofit=None, price=price_ask)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=None, takeprofit=None, price=price_bid)
print(f"Position: {id}")

# market orders all
volume=0.03
id = api.buyMarket(symbol, volume, stoploss=below, takeprofit=above, price=price_ask)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=above, takeprofit=below, price=price_bid)
print(f"Position: {id}")

# market orders with only SL
volume=0.02
id = api.buyMarket(symbol, volume, stoploss=below, takeprofit=None, price=None)
print(f"Position: {id}")
#id = api.sellMarket(symbol, volume, stoploss=above, takeprofit=None, price=None)
#print(f"Position: {id}")

# market orders with only TP
volume=0.03
id = api.buyMarket(symbol, volume, stoploss=None, takeprofit=above, price=None)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=None, takeprofit=below, price=None)
print(f"Position: {id}")



# market orders with both SL and TP, but without entry price
volume=0.05
id = api.buyMarket(symbol, volume, stoploss=below, takeprofit=above, price=None)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=above, takeprofit=below, price=None)
print(f"Position: {id}")

# market orders all
volume=0.06
id = api.buyMarket(symbol, volume, stoploss=below, takeprofit=above, price=price_ask)
print(f"Position: {id}")
id = api.sellMarket(symbol, volume, stoploss=above, takeprofit=below, price=price_bid)
print(f"Position: {id}")





In [ ]:
assert False


# market orders without price and without SL and TP
# market buy
volume=0.01
id = api.buy(symbol, volume, stoploss=None, takeprofit=None, price=None)
print(f"Position: {id}")
# market sell
# id = api.buy(symbol, volume, stoploss=above, takeprofit=below, price=None)
assert False

# market buy
id = api.buy(symbol, volume, stoploss=below, takeprofit=above, price=None)
print(f"Position: {id}")
# market sell
id = api.buy(symbol, volume, stoploss=above, takeprofit=below, price=None)
assert False
# market buy
id = api.buy(symbol, volume, stoploss=below, takeprofit=above, price=price_ask)
print(f"Position: {id}")
# market sell
id = api.buy(symbol, volume, stoploss=above, takeprofit=below, price=price_bid)
dfdfdfd
print(f"Position: {id}")
# limit order to buy
id = api.buyLimit(symbol, volume, price=limit_price_buy)
print(f"Order: {id}")
# limit order to sell
id = api.sellLimit(symbol, volume, price=limit_price_sell)
print(f"Order: {id}")
# stop order to buy
id = api.buyStop(symbol, volume, price=stop_price_buy)
print(f"Order: {id}")
# limit order to sell
id = api.sellStop(symbol, volume, price=stop_price_sell)
print(f"Order: {id}")

# Close all positions for one symbol

In [ ]:
api.positions()

In [ ]:
api.fix

In [ ]:
api.fix.spot_price_list

In [ ]:
api.fix.position_list_callback(api.fix.position_list, api.fix.spot_price_list, api.fix.client_id)

In [ ]:
api.fix.position_list

In [ ]:
api.fix.spot_price_list

In [ ]:
api.fix.client_id